<a href="https://colab.research.google.com/github/icarovazquez/agentic-ai/blob/main/Generic_Corporate_Strategy_Team.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **An Agentic AI Example: A Corporate Strategy Team**

## Team Introduction

We are a corporate strategy team inside a big US  company and we do research on any topic our manager needs. It is composed of the following members:


* A Corporate Strategy Manager whose job is to distribute tasks and collate the info from all the team members into a coherent whole   
* A Research Assistant whose job is to find information about the research topic
* A Technical Writer whose job is to write papers
* A Project Manager who takes the plan generated by the Corporate Strategy manager and assigns tasks to the other members of the team.

The team’s task is as follows:

Our manager will give us a research topic and we will do research online, write up a report of our findings, reflect on them and edit the report and finally, create a docx version of the report so it can be shared with our manager

In [1]:
#install needed libs
!pip install aisuite
!pip install anthropic
!pip install tavily-python
!pip install openai
!pip install pypandoc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: docstring-parser
    Found existing installation: docstring_parser 0.17.0
    Uninstalling docstring_parser-0.17.0:
      Successfully uninstalled docstring_parser-0.17.0
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
simple-parsing 0.1.7 requires docstring-parser<1.0,>=0.15, but you have docstring-parser 0.14.1 which is incompatible.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which is incompatible.
google-genai 1.49.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.
   

Next we import the required libs and setup a few tools the agents will have access to:


*   Tavily: to do web searches
*   Arxiv: to search academic papers
*   Wikipedia: for article searches





In [2]:
#import all the needed libs
import base64
import json
import os
import re
from datetime import datetime
from google.colab import userdata
from io import BytesIO
import pprint
import requests
import textwrap
import ast
import pypandoc #to create the docx version of the final report

# 3rd Party
from PIL import Image
from IPython.display import display, Image as ImageDisplay, IFrame, Markdown
import aisuite as ai
import urllib, urllib.request
import urllib.parse # Import urllib.parse
import openai
from tavily import TavilyClient
from google.colab import files

os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY').strip()


#initialize ai client
Client = ai.Client()

#Initializing research tools (tavily, arvix & wikipedia)

#initialize tavily client
tavily_client = TavilyClient(api_key=os.environ['TAVILY_API_KEY'])


#set arvix sample search term
arvix_search_term = 'impact of tariffs in the economy'
arvix_url = f'http://export.arxiv.org/api/query?search_query={urllib.parse.quote_plus(arvix_search_term)}&start=0&max_results=1' # Encode the search term


#set wikipedia url
wikipedia_search_url= 'https://api.wikimedia.org/core/v1/wikipedia/en/search/page'
headers = {
  'User-Agent': 'MyWikipediaApp/1.0 (icarovazquez@gmail.com)'
}

# We are going to use claude for this team
model="anthropic:claude-3-7-sonnet-20250219"

## **Tool Setup**

Now we run sample queries to make sure the tools work. First we run a query with tavily

In [3]:
#We ask Tavily to ask a simple question
search_response = tavily_client.search("Who is the chairman of the Federal Reserve Bank?")
pprint.pprint(search_response)


{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': 'Who is the chairman of the Federal Reserve Bank?',
 'request_id': 'b5773ca6-9671-4e94-9823-3337e35fca4f',
 'response_time': 1.42,
 'results': [{'content': '# Federal Reserve Chair * Federal Reserve Chair * '
                         'Federal Reserve Bank Leader * Federal Reserve Bank '
                         'of Atlanta * Federal Reserve Bank of Boston * '
                         'Federal Reserve Bank of Chicago * Federal Reserve '
                         'Bank of Cleveland * Federal Reserve Bank of Dallas * '
                         'Federal Reserve Bank of Kansas City * Federal '
                         'Reserve Bank of Minneapolis * Federal Reserve Bank '
                         'of New York * Federal Reserve Bank of Philadelphia * '
                         'Federal Reserve Bank of Richmond * Federal Reserve '
                         'Bank of San Francisco * Federal Reserve Bank of St. '
              

Now we run a query with arxiv

In [4]:
print(arvix_url)
data = urllib.request.urlopen(arvix_url)
print(data.read().decode('utf-8'))

http://export.arxiv.org/api/query?search_query=impact+of+tariffs+in+the+economy&start=0&max_results=1
<?xml version='1.0' encoding='UTF-8'?>
<feed xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/" xmlns:arxiv="http://arxiv.org/schemas/atom" xmlns="http://www.w3.org/2005/Atom">
  <id>https://arxiv.org/api/xKMIgR6WsVqcxt2UJY5CQDL8htA</id>
  <title>arXiv Query: search_query=all:impact OR all:of OR all:tariffs OR all:in OR all:the OR all:economy&amp;id_list=&amp;start=0&amp;max_results=1</title>
  <updated>2025-11-18T22:24:29Z</updated>
  <link href="https://arxiv.org/api/query?search_query=all:impact+OR+(all:of+OR+(all:tariffs+OR+(all:in+OR+(all:the+OR+all:economy))))&amp;start=0&amp;max_results=1&amp;id_list=" type="application/atom+xml"/>
  <opensearch:itemsPerPage>1</opensearch:itemsPerPage>
  <opensearch:totalResults>124269</opensearch:totalResults>
  <opensearch:startIndex>0</opensearch:startIndex>
  <entry>
    <id>http://arxiv.org/abs/2407.09487v1</id>
    <title>IT Enabling 

Now we try a wikipedia query

In [5]:
wikipedia_search_query = 'economic tariffs'
number_of_results = 10
parameters = {'q': wikipedia_search_query, 'limit': number_of_results}


wikipedia_response = requests.get(wikipedia_search_url, headers=headers, params=parameters)
wikipedia_data = wikipedia_response.json() # Use wikipedia_response instead of response
pprint.pprint(wikipedia_data)

{'pages': [{'anchor': None,
            'description': '',
            'excerpt': 'unprecedented <span class="searchmatch">tariff</span> '
                       'authority under the International Emergency <span '
                       'class="searchmatch">Economic</span> Powers Act '
                       '(IEEPA). On April 2, 2025, he invoked the law to '
                       'impose &quot;reciprocal <span '
                       'class="searchmatch">tariffs</span>&quot; on',
            'id': 78448911,
            'key': 'Tariffs_in_the_second_Trump_administration',
            'matched_title': None,
            'thumbnail': {'duration': None,
                          'height': 31,
                          'mimetype': 'image/svg+xml',
                          'url': '//upload.wikimedia.org/wikipedia/commons/thumb/f/fd/Map_of_countries_by_total_tariffs_in_the_second_Trump_administration.svg/60px-Map_of_countries_by_total_tariffs_in_the_second_Trump_administration.svg.png',
 

All 3 tools worked so that's great. However, that's not enough, we need to make them functions so they can be registered with aisuite and that's how we make them available to the LLM.

We start defining all 3 functions now.


In [6]:
# Tavily web search function
def tavily_search(web_query: str):
    """Search the web using Tavily."""
    search_response = tavily_client.search(web_query)
    # pprint.pprint(search_response) # Remove this to avoid printing raw response
    # data = search_response.json() # Tavily response is already a dictionary
    return search_response

# Arvix search function
def arvix_search(arvix_query: str):
    """Search academic papers with Arvix """
    arvix_url = f'http://export.arxiv.org/api/query?search_query={urllib.parse.quote_plus(arvix_query)}&start=0&max_results=1' # Encode the search term
    # print(arvix_url) # Remove this to avoid printing raw url
    data = urllib.request.urlopen(arvix_url)
    # print(data.read().decode('utf-8')) # Remove this to avoid printing raw response
    return data.read().decode('utf-8')


# Wikipedia search function
def wikipedia_search(wikipedia_query: str):
    """Search Wikipedia."""
    wikipedia_search_url= 'https://api.wikimedia.org/core/v1/wikipedia/en/search/page'
    number_of_results = 10
    parameters = {'q': wikipedia_query, 'limit': number_of_results}
    headers = {
      'User-Agent': 'MyWikipediaApp/1.0 (icarovazquez@gmail.com)'
    }

    wikipedia_response = requests.get(wikipedia_search_url, headers=headers, params=parameters)
    wikipedia_data = wikipedia_response.json() # Use wikipedia_response instead of response
    # pprint.pprint(wikipedia_data) # Remove this to avoid printing raw response
    return wikipedia_data

Now we are ready to define our team members (aka agents) and their tasks. Let's start with the Corporate Strategy Manager Agent


## **Corporate Strategy Manager Agent**
The first agent we define is the Corporate Strategy Manager. The Corporate Strategy Manager will generate the research plan that will answer the question posed by the user.

All the plan tasks will be executed by other members of the team.

In [8]:
def corporate_strategy_manager(research_topic):

  print("==================================")
  print("Corporate Strategy Manager Agent")
  print("==================================")

  user_prompt = f"""
    You are a Corporate Strategy manager whose job is to come up with a research plan that can be executed by other members of your team.
    Your team members are:

    1. A Research Assistant whose job is to find information about the research topic
    2. A Technical Writer whose job is summarize the research found by other agents
    3. An Editor whose job is to read summarize and modify them so they can distributed to the manager

    Your task is to create a step-by-step research plan as a valid Python list where each step is a string.

    Include real research-related tasks (e.g., search, summarize, draft, revise).
    Assume tool use is available.
    Do not include explanation text — return ONLY a Python list.
    The final step should be to generate a Markdown document containing the complete research report
    and should be assigned to the editor


    Here's your research topic: "{research_topic}"
  """

  messages = [{"role":"user", "content": user_prompt}]


  #Call the LLM and pass the research prompt
  llm_response = Client.chat.completions.create(
      model=model,
      messages=messages,
      temperature=1,
      )


  steps = llm_response.choices[0].message.content.strip()
  #print("The plan steps inside the corporate strategy manager are: ")
  #print(steps)

  cleaned_steps = re.sub(r"^```(?:python)?\s*|\s*```$", "", steps.strip(), flags=re.DOTALL)

  # Parse steps
  parsed_steps = ast.literal_eval(cleaned_steps)

  print(parsed_steps)

  return parsed_steps

Now we define the Research Assistant Agent.

## **The Research Assistant Agent**

The Research Assistant Agent's job is to do research on the topic requested by the user. It will use the tools available (web search, arxiv search & wikipedia search) and will generate research summaries

In [9]:
def research_agent(task: str, model: str = "openai:gpt-4o", return_messages: bool = False):
  """"
    Execute a research task using the available tools.
    Returns  the assistant text, or (text, messages) if return_messages=True.
    """
  print("==================================")
  print("🔍 Research Agent")
  print("==================================")

  current_time = datetime.now().strftime('%Y-%m-%d')

  res_prompt = f"""
    You are a research assistant who can search the web, wikipedia and arxiv.
    You can use arxiv_tool to find academic papers, tavily_tool
    for general web searches and wikipedia_tool for accessing encyclopedic
    knowledge

    Your task is {task}

    If needed, today's date is {current_time}
  """

  messages = [{"role": "user", "content": res_prompt}]

  # Save all of your available tools in the tools list.
  tools = [arvix_search,
          tavily_search,
          wikipedia_search]

  # Now we call the model with the tools
  response = Client.chat.completions.create(
      model=model,
      messages=messages,
      tools = tools,
      tool_choice="auto",
      max_turns=6
    )

  content = response.choices[0].message.content
  #print("LLM response for research agent is",content)

  if return_messages:
    return content, messages
  else:
    return content


Now we define the Technical Writer

## **Technical Writer**

The Technical Writer takes the research summaries created by the Research Assistant and generates a first version of the final report


In [10]:
def technical_writer_agent(task: str):

  print("==================================")
  print("Technical Writer Agent")
  print("==================================")

  #creates the writer prompt
  writer_prompt = f"""

  You are an expert technical writer that takes research summaries and generates
  technical content that can be consumed by company executives

  """

  print("the task the technical writer will execute is: ", task)


  # Define the system msg by using the system_prompt and assigning the role of system
  system_msg = {"role": "system", "content": writer_prompt}

  # Define the user msg. In this case the user prompt should be the task passed to the function
  user_msg = {"role": "user", "content": task}

  # Add both system and user messages to the messages list
  messages = [{"role": "system", "content": writer_prompt}, {"role": "user", "content": task}]



  #call the LLM
  writer_response = Client.chat.completions.create(
      model=model,
      messages=messages,
      temperature=1,
      )


  pprint.pprint(writer_response.choices[0].message.content)

  print("the full dump is: ")
  pprint.pprint(writer_response.__dict__)

  # Extract the content from the response

  content = writer_response.choices[0].message

  print(writer_response.choices[0].message)

  return content

Now we define the Editor Agent

## **Editor Agent**

The Editor takes the output from the Technical Writer agent, reflects on it, provides feedback on what could be improved and rewrites the report addressing the feedback it provided

In [11]:
def editor_agent(task: str):

  print("==================================")
  print("Editor Agent")
  print("==================================")

  editor_prompt = f"""

    You are an expert editor agent whose task is to reflect on, critique, and/or improve drafts
    """

  system_msg = {"role": "system", "content": editor_prompt}

  # We define the user message which includes the user prompt. The user prompt should be the task passed to the function
  user_msg = {"role": "user", "content": task}

  # Now we createa message with both prompts that we will pass to the LLM
  messages = [{"role": "system", "content": editor_prompt}, {"role": "user", "content": task}]

  response = Client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=1.0
    )

  return response.choices[0].message.content

We define a list of agents the Project Manager agent will use to assign tasks

In [12]:
list_of_agents = {
    "research_agent": research_agent,
    "editor_agent": editor_agent,
    "technical_writer_agent": technical_writer_agent,
    #"designer_agent": designer_agent
}

Finally, we define a Project Manager agent who manages all the agents and decides which agent will execute which task.

## **Project Manager Agent**

The Project Manager takes the plan generated by the Corporate Strategy manager and assigns tasks to the other agents.

In [13]:
def project_manager_agent(topic, model: str = "openai:gpt-4o", limit_steps: bool = True):

  print("==================================")
  print("Project Manager Agent")
  print("==================================")

  plan_steps = corporate_strategy_manager(topic)
  print("The plan steps from the corporate strategy manager are: ")
  pprint.pprint(plan_steps)
  max_steps = 15

  #this will hold a list of the tasks and agents that have already been executed
  history=[]

  for i, step in enumerate(plan_steps):

    agent_assignment_prompt = f"""
        You are an execution manager for a multi-agent research team.

        Given the instruction below, identify which agent should perform it and extract the clean task.

        Return only a valid JSON object with two keys:
        - "agent": one of ["research_agent", "editor_agent", "technical_writer_agent", "designer_agent"]
        - "task": a string with the instruction that the agent should follow

        Only respond with a valid JSON object. Do not include explanations or markdown formatting.

        Instruction: "{step}"

        """
    response = Client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": agent_assignment_prompt}],
        temperature=0,
        )


    raw_content = response.choices[0].message.content.strip()

    #cleaning up any markdown
    clean_json = re.sub(r"^```(?:json)?\s*|\s*```$", "", raw_content.strip())

    try:
      agent_content = json.loads(clean_json)
    except json.JSONDecodeError:
      print("⚠️ Could not decode JSON. Raw output:")
      print(raw_content)


    print("The formatted agent_content is ")
    pprint.pprint(agent_content)

    agent_name = agent_content["agent"]
    task_name = agent_content["task"]

    context = "\n".join([
            f"Step {j+1} executed by {a}:\n{r}"
            for j, (s, a, r) in enumerate(history)
        ])

    enriched_task = f"""
      You are {agent_name}.

      Here is the context of what has been done so far:
        {context}

      Your next task is:
        {task_name}

      """

    print(f"\n Agent: `{agent_name}` executing task: {enriched_task}")

    if agent_name in list_of_agents:
      output = list_of_agents[agent_name](enriched_task)
      history.append((step, agent_name, output))
    else:
      output = f" Unknown agent: {agent_name}"
      text_output = output.content if hasattr(output, "content") else str(output)
      history.append((step, agent_name, output))

    print(f"✅ Output:\n{output}")

  return history

And now we call the project manager agent who will execute the whole flow

In [14]:
research_topic = input("Please enter the topic you want this team to research:")
print(f"You entered the following: {research_topic}")


executor_history = project_manager_agent(research_topic, limit_steps=True)

md = executor_history[-1][-1].strip("`")
display(Markdown(md))

Streaming output truncated to the last 5000 lines.
      Here is the context of what has been done so far:
        Step 1 executed by research_agent:
Here's the information I found regarding Cisco's current business segments, product portfolio, and strategic focus areas as of 2025:

1. **Business Segments:**
   - Cisco operates business segments that include Networking, Security, Collaboration, and Cloud-based solutions. These segments cater to different technological needs and customer bases globally.

2. **Product Portfolio:**
   - Cisco's product portfolio consists of equipment and solutions related to Networking (e.g., switches, routers), Security (e.g., cybersecurity tools), Collaboration (e.g., communication and conferencing solutions), and Observability solutions, which help businesses monitor their IT environments effectively.

3. **Strategic Focus Areas:**
   - Cisco has a strategic focus on advancing its technologies in AI, security, and networking. They have been working on 

# Cisco Strategic Acquisition Analysis: Comprehensive Research Report

## Table of Contents

1. [Executive Summary](#executive-summary)
2. [Current Strategic Position](#current-strategic-position)
3. [Market Trends & Growth Opportunities](#market-trends--growth-opportunities)
4. [Competitive Landscape Analysis](#competitive-landscape-analysis)
5. [Potential Acquisition Targets](#potential-acquisition-targets)
6. [Target Evaluation Matrix](#target-evaluation-matrix)
7. [Top Recommendations](#top-recommendations)
8. [Integration Strategy](#integration-strategy)
9. [Financial Projections & ROI Analysis](#financial-projections--roi-analysis)
10. [Implementation Roadmap](#implementation-roadmap)
11. [Risk Assessment & Mitigation](#risk-assessment--mitigation)
12. [Appendices](#appendices)

---

## Executive Summary

Through 2030, technology markets will experience significant growth in several key areas:
- **Artificial Intelligence**: $2.6-4.4 trillion annual economic impact with over $200B in infrastructure spending by 2028
- **Data Center Infrastructure**: $6.7 trillion total investment projected through 2030
- **Healthcare Bio-Convergence**: Growing to $220.56B by 2030 (7.6% CAGR)
- **Clean Energy Technologies**: $2.2 trillion in investments by 2025

Cisco faces competitive challenges in brand recognition, pricing flexibility, and innovation velocity compared to both traditional competitors and cloud hyperscalers.

We recommend a phased acquisition strategy to address Cisco's competitive gaps, enhance core capabilities, and establish leadership in high-growth market segments:

### Phase 1 (Next 6 months)
- **DNSFilter** ($450-500M): Immediate enhancement of DNS security capabilities with minimal integration complexity and strong financial returns (18.7% IRR, 4-5 year payback)

### Phase 2 (6-12 months)
- **Moveworks** ($3.2-3.5B): Critical AI capability acquisition to enhance enterprise automation offerings and secure specialized AI engineering talent (12.5% IRR, 7-8 year payback)

### Phase 3 (12-24 months)
- **Mavenir** ($4.8-5.2B): Strategic positioning in 5G/6G infrastructure with focus on Open RAN and virtualized network functions (15.3% IRR, 5-6 year payback)
- **Samsara** ($16-18B): Established leader in industrial IoT with $1.4B revenue and proven profitability (14.8% IRR, 6-7 year payback)

### Phase 4 (24-36 months)
- **Arista Networks** ($65-70B): Transformational networking acquisition to consolidate market leadership and enhance AI infrastructure capabilities (11.2% IRR, 8-9 year payback)

This strategic acquisition program will position Cisco for sustained leadership through 2030 by accelerating AI capabilities, enhancing security offerings, establishing leadership in industrial IoT markets, and securing critical positioning in next-generation networking infrastructure.

---

## Current Strategic Position

### Core Business Segments and Product Portfolio

Cisco maintains a diversified business model across several key technology segments:

* **Core Business Segments**: Networking, Security, Collaboration, and Cloud-based solutions
* **Product Portfolio**: Spans hardware infrastructure (switches, routers), security solutions, collaboration tools, and observability platforms
* **Strategic Focus**: Concentrated on advancing AI integration, security enhancement, and next-generation networking

The company has strategically positioned itself at the intersection of critical enterprise technology needs, leveraging its historical strength in networking while expanding into high-growth adjacencies. Cisco's current strategy reflects an understanding that future enterprise value will be derived from integrated solutions rather than standalone products.

### Acquisition History (2019-2023)

Cisco has executed a targeted acquisition strategy focused on enhancing capabilities in high-growth areas:

#### Major Acquisitions

1. **Splunk (2023)** - $28 billion
   * Strategic rationale: Significantly expanded Cisco's observability and security capabilities
   * Integration focus: Enhancing Full Stack Observability, Extended Detection and Response (XDR), and Security Cloud platforms
   * Market impact: Positioned Cisco more competitively in the enterprise security and cloud protection space

2. **Duo Security** (Pre-2019, but strategically significant)
   * Strategic rationale: Strengthened identity management and zero-trust security capabilities
   * Market timing: Acquisition proved prescient as cloud application adoption and remote work accelerated
   * Integration outcome: Core component of Cisco's security portfolio as distributed workforce models expanded

3. **Code BGP (2023)** - Undisclosed amount
   * Strategic rationale: Enhanced BGP monitoring capabilities
   * Integration focus: Bolstering ThousandEyes network intelligence product suite
   * Technical value: Improved assessment of internet traffic health monitoring

#### Strategic Evolution

Cisco's acquisition trajectory demonstrates a deliberate pivot from hardware-centric to software and cloud-oriented capabilities. This transformation reflects market dynamics favoring recurring revenue models and cloud-delivered services over traditional infrastructure sales.

The company's M&A strategy reveals a clear understanding of technology market evolution, with particular emphasis on:

1. Cybersecurity enhancement
2. Software capabilities expansion
3. Cloud service delivery models
4. Observability and intelligence platforms

---

## Market Trends & Growth Opportunities

### Key Growth Sectors and Market Projections

#### Artificial Intelligence and Machine Learning
* **Economic Impact**: Generative AI expected to add $2.6-4.4 trillion annually to the global economy
* **Infrastructure Investment**: AI infrastructure spending projected to exceed $200 billion by 2028
* **Growth Driver**: Accelerated computing servers represent the primary investment category
* **Enterprise Adoption**: AI security platforms becoming essential for enterprise resilience in hyperconnected environments

#### Data Center and Computing Infrastructure
* **Total Investment Projection**: $6.7 trillion in next-generation compute and data center infrastructure by 2030
* **Strategic Importance**: Critical foundation for AI deployment, cloud services, and enterprise digital transformation
* **Growth Areas**: Specialized AI accelerators, sustainable computing solutions, and distributed infrastructure

#### Healthcare and Bio-Med Technology
* **Market Size Projection**: $220.56 billion by 2030 for healthcare bio-convergence market
* **CAGR**: Impressive 7.6% compound annual growth rate
* **Integration Opportunities**: Convergence of medical technologies with AI/ML capabilities creating novel solution categories

#### Clean Energy and Sustainable Technologies
* **Investment Trajectory**: Clean energy technologies expected to reach $2.2 trillion by 2025
* **Strategic Focus**: Sustainable computing and energy-efficient technologies gaining prominence
* **Industry Impact**: Transforming operational models across multiple sectors

### Competitive Landscape and Strategic Opportunities

#### Current Competitive Gaps for Cisco
* **Brand Recognition Challenges**: Limited recognition in certain emerging markets compared to competitors
* **Pricing Structure**: Premium pricing model creating competitive vulnerability against cost-effective alternatives
* **Product Category Competition**: Strong alternatives emerging in networking, security, and location services
* **Innovation Velocity**: Potential gaps in rapidly evolving markets where nimble startups can introduce disruptive solutions

#### Competitor Strategic Moves

##### Major Technology Players' Investment Strategies
* **Microsoft**: Committing approximately $80 billion in 2025 to AI infrastructure and workloads
* **Google**: Part of tech megacap cohort spending $320+ billion on AI technology development in 2025
* **Amazon/AWS**: Strategically acquiring companies aligned with cloud expansion and AI platform enhancement

##### Common Strategic Patterns
* **AI Integration**: All major competitors are embedding AI capabilities across their product portfolios
* **Cloud Infrastructure Scaling**: Significant capital expenditure to support growing AI compute demands
* **Security Enhancement**: Increasing focus on integrated security solutions for complex digital environments
* **Acquisition Focus**: Strategic acquisitions targeting specialized capabilities in AI, cloud, and security

---

## Competitive Landscape Analysis

### Microsoft
- **Focus Areas**: Microsoft is integrating AI technologies across its services, placing emphasis on cloud computing and AI-driven solutions. It's committed to expanding the role of AI in products like Microsoft Copilot.
- **Acquisition Strategy**: Microsoft has been actively acquiring companies that bolster its cloud services (Azure) and AI capabilities, expanding their enterprise and consumer product suite.
- **Investments**: Microsoft plans to spend approximately $80 billion in 2025 focusing on AI infrastructure and workloads data centers.

### Amazon
- **Focus Areas**: Amazon, particularly through AWS, is prioritizing the development and deployment of secure, accessible AI services for developers and businesses, expanding generative AI capabilities.
- **Acquisition Strategy**: Amazon strategically acquires companies that align with its cloud expansion goals and can enhance its AI platforms like Amazon Bedrock.
- **Investments**: In the fiscal year 2025, Amazon is set to invest heavily in AI technologies, contributing to a cumulative $364 billion investment in the sector.

### Google (Alphabet)
- **Focus Areas**: Google aims to embed AI into every product it offers, catering to both consumer and enterprise needs, enhancing user engagement across product lines.
- **Acquisition Strategy**: Google's acquisition efforts are centered on AI-centric firms, focusing on boosting its technological footprints in cloud services and search engine enhancements.
- **Investments**: Google is part of a concerted move by tech megacaps to spend upwards of $320 billion in AI technology development in 2025.

### General Trends in Competitor Strategies
- **AI Dominance**: All three companies are significantly increasing their AI investments, reflecting a strategic alignment towards technology spaces where AI, cloud computing, and enhanced cybersecurity solutions are intertwined.
- **Cloud Computing**: Scaling cloud infrastructure to support AI workloads and ensuring secure, scalable service offerings remains central to each company's strategy.
- **Market Adaptation**: As the technology landscape evolves, these giants are strategically positioned to make acquisitions that ensure competitiveness in innovative tech such as AI, cloud services, and IoT.

---

## Potential Acquisition Targets

### AI and Machine Learning Targets

#### 1. DeepL
**Financial Profile:**
- Revenue Growth: 13.9% over past three years
- Valuation: Approaching $5 billion
- IPO Plans: U.S. listing anticipated

**Strategic Rationale:**
DeepL's advanced language translation technology would provide Cisco with best-in-class AI capabilities that could be integrated across collaboration platforms, enabling seamless global communication for enterprise customers. This acquisition would significantly enhance Cisco's international market positioning and address the growing demand for AI-powered multilingual business tools.

#### 2. Moveworks
**Financial Profile:**
- Annual Revenue: $100 million (2024)
- Valuation: $2.8 billion
- Growth Rate: 9.4% annually

**Strategic Rationale:**
Moveworks' AI-powered enterprise ticketing and IT support automation capabilities would allow Cisco to expand its enterprise service management offerings. This acquisition would address competitive gaps in Cisco's automation portfolio while providing entry into the rapidly growing IT service management AI market. Integration with Cisco's existing enterprise solutions would create a comprehensive, AI-driven business operations platform.

#### 3. DNSFilter
**Financial Profile:**
- Estimated Annual Revenue: $38.4 million
- Growth Rate: 800% over five years

**Strategic Rationale:**
DNSFilter's AI-powered DNS security technology would complement Cisco's existing security portfolio, addressing the growing threat landscape from malicious domains. This acquisition would strengthen Cisco's zero-trust security architecture and provide advanced threat intelligence capabilities. The integration would deliver enhanced protection for Cisco's networking customers while expanding recurring revenue streams.

### Cybersecurity Leaders

#### 4. ID.me
**Financial Profile:**
- Total Funding: $340 million
- Valuation: Exceeds $2 billion

**Strategic Rationale:**
ID.me's digital identity verification platform would provide Cisco with advanced authentication capabilities essential for securing hybrid work environments. This acquisition would enhance Cisco's zero-trust security framework by adding sophisticated identity management tools. The technology would integrate seamlessly with Cisco's existing Duo Security platform, creating a comprehensive identity solution for enterprise customers.

### Cloud and Infrastructure Innovators

#### 5. DigitalOcean
**Financial Profile:**
- Projected Annual Revenue: $896-897 million
- AI Business Growth: Over 100% year-over-year

**Strategic Rationale:**
DigitalOcean's SMB-focused cloud platform would give Cisco immediate access to a new customer segment currently underserved by its enterprise focus. This acquisition would provide a complementary cloud infrastructure offering that addresses competitive pricing challenges while leveraging Cisco's networking expertise. The company's strong AI growth aligns with Cisco's strategic priorities.

#### 6. Vercel
**Financial Profile:**
- Annual Revenue: $200 million
- Valuation: $8-9 billion
- Total Funding: $563 million

**Strategic Rationale:**
Vercel's frontend cloud platform would significantly enhance Cisco's developer ecosystem, providing tools for building and deploying modern web applications. This acquisition would position Cisco competitively in the rapidly growing market for cloud-native application development. The integration would create synergies with Cisco's cloud infrastructure and API management capabilities.

### IoT and Operational Technology

#### 7. TRACTIAN
**Financial Profile:**
- Estimated Annual Revenue: $109.1 million
- Total Funding: $196 million

**Strategic Rationale:**
TRACTIAN's industrial asset monitoring and predictive maintenance platform would strengthen Cisco's IoT portfolio with specialized capabilities for manufacturing and industrial customers. This acquisition would accelerate Cisco's industrial IoT strategy while providing entry into the high-growth predictive maintenance market. The technology would complement Cisco's edge computing and industrial networking solutions.

#### 8. Samsara
**Financial Profile:**
- Annual Revenue: $1.4 billion
- EBITDA: $210 million

**Strategic Rationale:**
Samsara's connected operations cloud would give Cisco a comprehensive IoT platform serving transportation, logistics, and field service industries. This acquisition would significantly expand Cisco's IoT capabilities and recurring revenue streams. Samsara's proven profitability and growth trajectory make it an attractive target despite its size, offering immediate scale in IoT markets.

### 5G/6G and Network Innovation

#### 9. Mavenir
**Financial Profile:**
- Total Funding: $1.74 billion
- Focus: Open RAN and core network solutions

**Strategic Rationale:**
Mavenir's Open RAN expertise would strengthen Cisco's 5G/6G capabilities, positioning the company competitively in next-generation wireless infrastructure. This acquisition would accelerate Cisco's telecommunications strategy while addressing competitive pressures from traditional telecom equipment providers. Mavenir's software-centric approach aligns with Cisco's strategic direction.

#### 10. Arista Networks
**Financial Profile:**
- Q3 Revenue: $2.31 billion
- Growth Rate: 27.5% year-over-year

**Strategic Rationale:**
While a substantial acquisition, Arista's network hardware capabilities would significantly strengthen Cisco's data center networking portfolio, particularly for AI workloads. This acquisition would consolidate Cisco's position in high-performance networking while eliminating a key competitor. Arista's technical innovation and growth trajectory would provide immediate value to Cisco's core networking business.

---

## Target Evaluation Matrix

This decision matrix provides a structured approach for evaluating potential acquisition targets based on key strategic, financial, and operational criteria. Each criterion is assigned a weight reflecting its importance to Cisco's strategic objectives.

### Evaluation Framework

| Evaluation Criteria | Weight | Description |
|---------------------|--------|-------------|
| **Strategic Alignment** | 25% | Alignment with Cisco's core focus areas (networking, security, cloud, AI) |
| **Financial Performance** | 20% | Revenue growth, profitability, and valuation considerations |
| **Technology Differentiation** | 15% | Uniqueness and competitive advantage of target's technology |
| **Market Position** | 10% | Market share, brand strength, and competitive landscape |
| **Integration Complexity** | 10% | Ease of technical, cultural, and operational integration |
| **Talent Acquisition** | 10% | Quality and depth of engineering and leadership talent |
| **Regulatory Risk** | 5% | Potential regulatory hurdles and compliance considerations |
| **Customer Synergies** | 5% | Complementary customer base and cross-selling opportunities |

### Target Evaluation Matrix

| Acquisition Target | Strategic Alignment<br>(25%) | Financial Performance<br>(20%) | Technology Differentiation<br>(15%) | Market Position<br>(10%) | Integration Complexity<br>(10%) | Talent Acquisition<br>(10%) | Regulatory Risk<br>(5%) | Customer Synergies<br>(5%) | Total Score |
|-------------------|------------------|----------------------|--------------------------|----------------|------------------------|---------------------|-----------------|---------------------|------------|
| **DeepL** | 8 (2.00) | 7 (1.40) | 9 (1.35) | 7 (0.70) | 6 (0.60) | 8 (0.80) | 7 (0.35) | 6 (0.30) | **7.50** |
| **Moveworks** | 9 (2.25) | 7 (1.40) | 8 (1.20) | 6 (0.60) | 7 (0.70) | 8 (0.80) | 8 (0.40) | 9 (0.45) | **7.80** |
| **DNS

Now that we have the final report and give the user the option of downloading it in docx (that both Google Docs or Word can open)

In [15]:
print(md)

# Cisco Strategic Acquisition Analysis: Comprehensive Research Report

## Table of Contents

1. [Executive Summary](#executive-summary)
2. [Current Strategic Position](#current-strategic-position)
3. [Market Trends & Growth Opportunities](#market-trends--growth-opportunities)
4. [Competitive Landscape Analysis](#competitive-landscape-analysis)
5. [Potential Acquisition Targets](#potential-acquisition-targets)
6. [Target Evaluation Matrix](#target-evaluation-matrix)
7. [Top Recommendations](#top-recommendations)
8. [Integration Strategy](#integration-strategy)
9. [Financial Projections & ROI Analysis](#financial-projections--roi-analysis)
10. [Implementation Roadmap](#implementation-roadmap)
11. [Risk Assessment & Mitigation](#risk-assessment--mitigation)
12. [Appendices](#appendices)

---

## Executive Summary

Through 2030, technology markets will experience significant growth in several key areas:
- **Artificial Intelligence**: $2.6-4.4 trillion annual economic impact with over $200B

Now that we have the final report, we produce the docx download option

In [16]:
output_path = "research_report.docx"


#we convert the markdown to docx
pypandoc.convert_text(
    md,
    to="docx",
    format="md",
    outputfile=output_path
)

''

and now we can download the report in the format requested.

In [17]:
files.download("research_report.docx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>